In [19]:
from __future__ import print_function
import pickle, os.path, json
from googleapiclient.errors import HttpError
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [20]:
import json
import requests

In [21]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = [
          'https://www.googleapis.com/auth/classroom.courses',
          'https://www.googleapis.com/auth/classroom.coursework.students',
          'https://www.googleapis.com/auth/classroom.coursework.me'
         ] 

In [22]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# Store on obj
service = build('classroom', 'v1', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=977164804267-d79kh905vrdeqjata0iuh33ocsu9oukc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.courses+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.coursework.students+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.coursework.me&state=8Ry9uYJtQQaefCV67sWmioJbG9psPJ&access_type=offline


In [124]:
def get_all_courses():
    # Call the Classroom API
    list_of_courses = {}
    results = service.courses().list(pageSize=10,).execute()
    courses = results.get('courses', [])

    #print(results)
    f = open('courses.json', 'w+')
    f.write(json.dumps(results))
    f.close()

    if not courses:
        print('No courses found.')
    else:
        print('Courses:')
        for course in courses:
            list_of_courses[course['name']] = course['id']

    return list_of_courses

In [125]:
courses = get_all_courses()
course_id = courses['Test Class']

print(course_id)

Courses:
36784271501


In [127]:
def get_all_assignments(course_id):
    #get all courseWork for Course
    list_of_assignments = {}
    courseWorks = service.courses().courseWork().list(
        courseId=course_id).execute()
    courseWork = courseWorks.get('courseWork', [])

    if not courseWork:
        print('No courseWork found.')
    else:
        print('courseWork:')
        for work in courseWork:
            list_of_assignments[work['title']] = work['id']
            
    return list_of_assignments

In [132]:
assignments = get_all_assignments(course_id)
assignment_id = assignments['Assignment 1 - Make Pong Interface']

print(assignment_id)

courseWork:
36791124697


In [133]:
# Get assignment submissions
assignment_submissions = {}
submissions = service.courses().courseWork().studentSubmissions().list(
    courseId=course_id, 
    courseWorkId=assignment_id,
).execute()

studentSubmissions = submissions.get('studentSubmissions', [])

if not studentSubmissions:
    print('No studentSubmissions found.')
else:
    print('studentSubmissions:')
    for studentSubmission in studentSubmissions:
        print(studentSubmission['id'], studentSubmission['userId'])

studentSubmissions:
Cg0IpsDPnSkQ2YWwh4kB 110544833030086357366
Cg0IsYmxkGwQ2YWwh4kB 111563391117804030607


In [136]:
# Get a student's assignment submissions
submissions = service.courses().courseWork().studentSubmissions().list(
    courseId=course_id, 
    courseWorkId=assignment_id,
).execute()

print(submissions)
studentSubmissions = submissions.get('studentSubmissions', [])

if not studentSubmissions:
    print('No studentSubmissions found.')
else:
    print('studentSubmissions:')
    for studentSubmission in studentSubmissions:
        print(studentSubmission['id'], studentSubmission['userId'])

{'studentSubmissions': [{'courseId': '36784271501', 'courseWorkId': '36791124697', 'id': 'Cg0IpsDPnSkQ2YWwh4kB', 'userId': '110544833030086357366', 'creationTime': '2019-05-17T15:42:53.684Z', 'updateTime': '2019-05-17T15:42:53.673Z', 'state': 'CREATED', 'alternateLink': 'https://classroom.google.com/c/MzY3ODQyNzE1MDFa/a/MzY3OTExMjQ2OTda/submissions/student/MTEwNjc5NzM2NzBa', 'courseWorkType': 'ASSIGNMENT', 'assignmentSubmission': {}, 'associatedWithDeveloper': True, 'submissionHistory': [{'stateHistory': {'state': 'CREATED', 'stateTimestamp': '2019-05-17T15:42:53.673Z', 'actorUserId': '110544833030086357366'}}]}, {'courseId': '36784271501', 'courseWorkId': '36791124697', 'id': 'Cg0IsYmxkGwQ2YWwh4kB', 'userId': '111563391117804030607', 'creationTime': '2019-05-17T15:42:53.684Z', 'updateTime': '2019-05-17T15:42:53.673Z', 'state': 'CREATED', 'alternateLink': 'https://classroom.google.com/c/MzY3ODQyNzE1MDFa/a/MzY3OTExMjQ2OTda/submissions/student/MjkwMjUzODc2OTda', 'courseWorkType': 'ASSIGN

In [134]:
def send_results(course_id, assignment_id, submission_id, grade):
    # grade an assignment
    reqs =  {
                'assignedGrade': grade,
                'draftGrade': grade
            } 

    submissions = service.courses().courseWork().studentSubmissions().patch(
        courseId=course_id, 
        courseWorkId=assignment_id, 
        id=submission_id, 
        updateMask = 'assignedGrade,draftGrade', 
        body = reqs,
    ).execute()

    print('success')

In [118]:
#creating a course
course_details = {
  "title": "Assignment 1 - Make Pong Interface",
  "materials": [
    {
      "driveFile": {
        "driveFile":{
          'id': '1uNwWfBI9-JTlcmh4OZ8W9s7xPc9EuoupEIx6inne9iQ',
          "title": 'Assignment 1 - Make Pong Interface',
          "alternateLink": 'https://drive.google.com/open?id=1uNwWfBI9-JTlcmh4OZ8W9s7xPc9EuoupEIx6inne9iQ'
        },
        'shareMode': 'VIEW'
      },
    }
  ],
  "state": 'PUBLISHED',
  "dueDate": {
    "year": 2019,
    "month": 5,
    "day": 18
  },
  "dueTime": {
    "hours": 18,
    "minutes": 00,
    "seconds": 00,
  },
  "maxPoints": 20,
  "workType": 'ASSIGNMENT',
  "assigneeMode": 'ALL_STUDENTS',
  "creatorUserId": 'me',

#   "assignment": {
#     object(Assignment)
#   },
}
    
#create coursework/assignment
submissions = service.courses().courseWork().create( 
    courseId = course_id,
    body = course_details
).execute()